In [53]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

batch_size = 32  # Batch size for training.
epochs = 200  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 12937  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = r'Transliteration\NEWS2018_DATASET_04/NEWS2018_M-EnHi_trn.txt'
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
lines


['aabhaa\tआभा\n',
 'aabheer\tआभीर\n',
 'aabhijaat\tआभिजात\n',
 'aabid\tआबिद\n',
 'aabshar\tआबशर\n',
 'aachaaryanandan\tआचार्यनंदन\n',
 'aachaaryanandanaa\tआचार्यनंदना\n',
 'aachaaryasut\tआचार्यसुत\n',
 'aachaaryasuta\tआचार्यसुता\n',
 'aachaaryasutaa\tआचार्यसुता\n',
 'aachaarynandan\tआचार्यनंदन\n',
 'aachaarysuta\tआचार्यसुता\n',
 'aachaarysutaa\tआचार्यसुता\n',
 'aacharya\tआचार्य\n',
 'aacharyanandana\tआचार्यनंदना\n',
 'aacharyanandanaa\tआचार्यनंदना\n',
 'aacharyasuta\tआचार्यसुता\n',
 'aacharyasutaa\tआचार्यसुता\n',
 'aacharynandana\tआचार्यनंदना\n',
 'aacharysut\tआचार्यसुत\n',
 'aachrekar\tआचरेकर\n',
 'aachwal\tआचवल\n',
 'aada\tआडा\n',
 'aadarsh\tआदर्श\n',
 'aadarsha\tआदर्श\n',
 'aadat se majboor\tआदत से मजबूर\n',
 'aadesh\tआदेश\n',
 'aadha din aadhi raat\tआधा दिन आधी रात\n',
 'aadhi raat\tआधी रात\n',
 'aadhi yug\tआधी युग\n',
 'aadhunik\tआधुनिक\n',
 'aadikavi\tआदिकवि\n',
 'aadinaaraayan\tआदिनारायण\n',
 'aadinaraayan\tआदिनारायण\n',
 'aadinath\tआदिनाथ\n',
 'aadishvar\tआदिश्वर\n',
 'aaditya\

In [54]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

print(input_characters) 
print(target_characters) 

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 12937
Number of unique input tokens: 43
Number of unique output tokens: 85
Max sequence length for inputs: 61
Max sequence length for outputs: 64
[' ', "'", '(', ')', ',', '-', '.', '/', '1', '2', '4', '6', '?', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'è', 'é']
['\t', '\n', ' ', "'", '(', ')', ',', '-', '.', '/', '1', '2', '4', '6', ':', '?', 'ँ', 'ं', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'क़', 'ख़', 'ग़', 'ज़', 'ड़', 'ढ़', 'फ़', '॥', '\u200d']


In [55]:
#Build the model
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Train the model
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1,
)
# Save model
model.save(r'/content/drive/MyDrive/Colab Notebooks/Transliteration/s2s_google_1')

Epoch 1/200
364/364 [==============================] - 14s 17ms/step - loss: 0.6390 - accuracy: 0.8602 - val_loss: 0.5576 - val_accuracy: 0.8624
Epoch 2/200
364/364 [==============================] - 4s 11ms/step - loss: 0.4640 - accuracy: 0.8758 - val_loss: 0.5139 - val_accuracy: 0.8678
Epoch 3/200
364/364 [==============================] - 4s 11ms/step - loss: 0.4360 - accuracy: 0.8803 - val_loss: 0.4964 - val_accuracy: 0.8732
Epoch 4/200
364/364 [==============================] - 5s 14ms/step - loss: 0.4214 - accuracy: 0.8827 - val_loss: 0.4912 - val_accuracy: 0.8746
Epoch 5/200
364/364 [==============================] - 4s 11ms/step - loss: 0.4109 - accuracy: 0.8847 - val_loss: 0.4826 - val_accuracy: 0.8768
Epoch 6/200
364/364 [==============================] - 4s 11ms/step - loss: 0.4003 - accuracy: 0.8875 - val_loss: 0.4760 - val_accuracy: 0.8788
Epoch 7/200
364/364 [==============================] - 5s 14ms/step - loss: 0.3910 - accuracy: 0.8897 - val_loss: 0.4724 - val_accuracy

In [56]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model(r'/content/drive/MyDrive/Colab Notebooks/Transliteration/s2s_google_1')

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [57]:
#pip show tensorflow

In [58]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)
    

1/1 [==============================] - 0s 18ms/step
-
Input sentence: aabhaa
Decoded sentence: आभा

1/1 [==============================] - 0s 19ms/step
-
Input sentence: aabheer
Decoded sentence: आभीर

1/1 [==============================] - 0s 19ms/step
-
Input sentence: aabhijaat
Decoded sentence: आभिजात

1/1 [==============================] - 0s 19ms/step
-
Input sentence: aabid
Decoded sentence: आबिद

1/1 [==============================] - 0s 20ms/step
-
Input sentence: aabshar
Decoded sentence: आबशर

1/1 [==============================] - 0s 18ms/step
-
Input sentence: aachaaryanandan
Decoded sentence: आचार्यनंदन

1/1 [==============================] - 0s 28ms/step
-
Input sentence: aachaaryanandanaa
Decoded sentence: आचार्यनंदना

1/1 [==============================] - 0s 33ms/step
-
Input sentence: aachaaryasut
Decoded sentence: आचार्यसुत

1/1 [==============================] - 0s 29ms/step
-
Input sentence: aachaaryasuta
Decoded sentence: आचार्यसुता

1/1 [========================